# **Tidying**

Name: Haysam

Part of the data engineering process is data cleaning and tidying. What is done in those two processes, is trying to make the data more readable, and complete. This makes much easier to analyze, visualize, and train the data.


## **Data Tidying**

Making the data more organized, and readable is the result of applying data tidying. 

In this section two main pandas functions are used in data tidying those are `melt` and `pivot_table`.

Let's start by taking a look at the below dataframe, which represents the income ranges based on religion. This is part of the PEW research, which is famous in the US for conducting pollings and surveys on citizens.

When the following are satisfied:


1. Each variable forms a column
2. Each observation forms a row
3. Each type of observational unit forms a table

We can then say that our dataset is *tidy*.

First we need to import pandas to read csv datasets.

In [170]:
# importing dependencies
import pandas as pd
import numpy as np


### **PEW Research Dataset**

1. Start by Importing the dataset into a pandas dataframe.

In [171]:
# loding the CSV file into a pandas dataframe #
path1 = 'CSVs/pew-raw.csv' # setting the path to the CSV file
df1 = pd.read_csv(path1)
#printing the first 5 rows of the dataframe
print(df1.head() ,'\n') 

            religion   <$10k   $10-20k  $20-30k  $30-40k   $40-50k  $50-75k
0           Agnostic      27        34       60       81        76      137
1            Atheist      12        27       37       52        35       70
2           Buddhist      27        21       30       34        33       58
3           Catholic     418       617      732      670       638     1116
4  Dont know/refused      15        14       15       11        10       35 



In [172]:
print('#'*20, 'Columns Name', '#' * 20) # viewing the columns names
print(df1.columns)
print('#'*20, 'Data shape', '#' * 20) # viewing the columns names
print(df1.shape)

#################### Columns Name ####################
Index(['religion', ' <$10k', ' $10-20k', '$20-30k', '$30-40k', ' $40-50k',
       '$50-75k'],
      dtype='object')
#################### Data shape ####################
(10, 7)


2. Observe the dataset using the `loc`, `iloc`, `head`, or `tail` approaches

In [173]:
print(df1.loc[0,' $10-20k'])  # Display row with index '$20-30k'
print(df1.iloc[2:5]) # Display rows 2 to 4


34
            religion   <$10k   $10-20k  $20-30k  $30-40k   $40-50k  $50-75k
2           Buddhist      27        21       30       34        33       58
3           Catholic     418       617      732      670       638     1116
4  Dont know/refused      15        14       15       11        10       35


In [174]:
print('#'*20, 'tail', '#' * 20)
print(df1.tail())  # Display last 5 rows

#################### tail ####################
                   religion   <$10k   $10-20k  $20-30k  $30-40k   $40-50k  \
5         Evangelical Prot      575       869     1064      982       881   
6                    Hindu        1         9        7        9        11   
7  Historically Black Prot      228       244      236      238       197   
8         Jehovahs Witness       20        27       24       24        21   
9                   Jewish       19        19       25       25        30   

   $50-75k  
5     1486  
6       34  
7      223  
8       30  
9       95  


***What does not seem right in the above dataframe?***
- Some column names contain leading spaces ' $10-20k'
- The data is understandable to humans but not suitable for data processing.
- Each variable must have its own column
- Each observation must have its own row
- Each value must have its own cell 


3. Try to make the column headers represent a variable not a value. For that, use the `melt` function.

In [175]:
df_tidy = df1.melt(id_vars="religion", var_name="income", value_name="count")
print(df_tidy.head())  # Display first 5 rows of the tidy dataframe

            religion  income  count
0           Agnostic   <$10k     27
1            Atheist   <$10k     12
2           Buddhist   <$10k     27
3           Catholic   <$10k    418
4  Dont know/refused   <$10k     15


**Converted income range strings into numerical min_income and max_income values to enable numerical analysis**

In [176]:
# Function to extract min and max income values
def parse_income_range(income_str):
    income_str = income_str.strip()
    if income_str.startswith('<$'):
        return (0, 10000)
    elif '-' in income_str:
        parts = income_str.replace('$', '').replace('k', '').split('-')
        return (int(parts[0]) * 1000, int(parts[1]) * 1000)
    elif income_str.startswith('>$'):
        return (100000, np.nan)  # optional case
    else:
        return (np.nan, np.nan)

# Apply parsing to income column
df_tidy[['min_income', 'max_income']] = df_tidy['income'].apply(
    lambda x: pd.Series(parse_income_range(x))
)

# Show result
print(df_tidy)  # Display rows 5 to 9 of the tidy dataframe with income ranges

                    religion    income  count  min_income  max_income
0                   Agnostic     <$10k     27           0       10000
1                    Atheist     <$10k     12           0       10000
2                   Buddhist     <$10k     27           0       10000
3                   Catholic     <$10k    418           0       10000
4          Dont know/refused     <$10k     15           0       10000
5          Evangelical Prot      <$10k    575           0       10000
6                     Hindu      <$10k      1           0       10000
7   Historically Black Prot      <$10k    228           0       10000
8          Jehovahs Witness      <$10k     20           0       10000
9                    Jewish      <$10k     19           0       10000
10                  Agnostic   $10-20k     34       10000       20000
11                   Atheist   $10-20k     27       10000       20000
12                  Buddhist   $10-20k     21       10000       20000
13                  

In [177]:
df_tidy.drop(columns='income', inplace=True) # Drop the original income column
print(df_tidy.head(),'\n')  # Display first 5 rows of the tidy dataframe after dropping the income column
print('#'*20, 'Data types', '#' * 20)  # Displaying the data types of the columns in the tidy dataframe
print('checking the data types of the columns in the tidy dataframe')
print(df_tidy.dtypes)  # Display the data types of the columns in the tidy dataframe

            religion  count  min_income  max_income
0           Agnostic     27           0       10000
1            Atheist     12           0       10000
2           Buddhist     27           0       10000
3           Catholic    418           0       10000
4  Dont know/refused     15           0       10000 

#################### Data types ####################
checking the data types of the columns in the tidy dataframe
religion      object
count          int64
min_income     int64
max_income     int64
dtype: object


### **Billboard Dataset**

This dataset outlines data about the top hit songs on the Billboard list and the week from entrance that it was in the billboard with the ranking.

1. Read the dataset and store it in a pandas dataframe. Note that the usual utf-8 encoding does not work on this dataset. The reason behind this is that there might be characters that are not supported by `utf-8`.

The suggestion is to use for this dataset `unicode_escape` encoding. (converts all non-ASCII characters into their \uXXXX representations)

2. Observe the first few rows of the dataset.

***What is wrong with the above dataset?***

3. Let's, again, use the `melt` function to fix the general structure of the dataframe.

If we inspect the current dataframe. We find that it is structured in a better way than before. 

However, the ***Week*** column looks a bit ugly!

4. Let's try to place only the week number in that column without the extras surronding it.

5. Now let's inspect the ***Week*** column in the dataframe.

Next, let's try to find the date at which the song ranked the number that is shown per row.

6. To do that let's first think of the equation that is going to get us the relevant date at which the song ranked the *rth*.



*Timedeltas are absolute differences in times, expressed in difference units (e.g. days, hours, minutes, seconds). This method converts an argument from a recognized timedelta format / value into a Timedelta type.*


***What is the problem with the calculation above?***

7. Let's only keep necessary columns

8. How to rename your columns?

Display the dataframe

In the above dataframe, there are some *NaN* values. What are we going to do? <br/>
9. Apply quick data cleaning and then observe the dataset

## **Data Cleaning**

Data cleaning involves removing unwanted characters, imputing, or dropping missing values.

The decision is based on the dataset you have, and the information you can extract from the other columns.


Examples of data cleaning include cleaning:

1.   **Missing Data**
2.   **Irregular Data** (Outliers)
3.   **Unnecessary Data** — Repetitive Data, Duplicates and more
4.   **Inconsistent Data** — Capitalization, Addresses and more








### **Cars Data Set**

Start by reading the dataset related to car models: ./CSVs/cars.csv

First Row seems to be the datatype, we need to remove it

Let's observe the columns with null values.  Either by using the `isnull().sum()` function 

There aren't many missing values. Let's take a glimpse at the percentage of the missing values:

**HINT:** We'll need `Numpy` for the below task.

Around *0.19%* of the values are missing, which isn't a lot. Therefore, we might go with the option of dropping all the rows with null values.

Lets also check dropping the columns

Let's observe how many columns we lost

### **Cars Dataset - Filling in missing values automatically**

Another option is to try and fill in the missing values through imputations.

Let's take the `MPG` column for example. We can fill in the missing values with 0s through the following line of code:

`df_cars.fillna(0) `. 

However, this does not make much sense as there isn't MPG equal to 0. How about we plot the MPG column and if it follows a random distribution we can use the mean of the column to compute the missing values. Otherwise, we can use the median (if there is a skewed normal distribution). However, there might be a better way of imputation which is getting the median or the mean of the MPG of the cars with similar attributes.

If we observe the graph above, we can consider it in a way or another normally distributed. Therefore, we can impute the missing values using the mean.

To compute the mean we need numeric values. However the values in the dataframe are objects. Therefore, we need to change them to numerics so that we can compute them.

Now let's see what is the mean of the MPG column

We can use this mean to compute the missing values since the graph demonstarted a normal distribution

### **Car Dataset - Simple Imputer**

*SimpleImputer* is a `scikit-learn` class which is helpful in handling the missing data in the predictive model dataset. It replaces the `NaN` values with a specified placeholder.
It is implemented by the use of the `SimpleImputer()` method which takes the following arguments :

`missing_values` : The missing_values placeholder which has to be imputed. By default is NaN

`strategy` : The data which will replace the NaN values from the dataset. The strategy argument can take the values – ‘mean'(default), ‘median’, ‘most_frequent’ and ‘constant’.


Let's start by importing the `SimpleImputer` into our notebook

What we need to do are two essential steps:

1. fit the data (compute the mean / median / most freq)
2. transform the data (place the computed values in the NaN cells)

## **Outlier Detection** 


An Outlier is a data-item/object that deviates significantly from the rest of the (so-called normal)objects. They can be caused by measurement or execution errors. The analysis for outlier detection is referred to as outlier mining. There are many ways to detect the outliers, and the removal process is the data frame same as removing a data item from the panda’s data frame.



https://www.geeksforgeeks.org/detect-and-remove-the-outliers-using-python/





### Outliers Visualization

#### Visualizing Outliers Using Box Plot
It captures the summary of the data effectively and efficiently with only a simple box and whiskers. Boxplot summarizes sample data using 25th, 50th, and 75th percentiles. One can just get insights(quartiles, median, and outliers) into the dataset by just looking at its boxplot.

#### Visualizing Outliers Using ScatterPlot.

It is used when you have paired numerical data and when your dependent variable has multiple values for each reading independent variable, or when trying to determine the relationship between the two variables. In the process of utilizing the scatter plot, one can also use it for outlier detection.

#### Z-Score:
Z- Score is also called a standard score. This value/score helps to understand that how far is the data point from the mean. And after setting up a threshold value one can utilize z score values of data points to define the outliers.
<br>
Zscore = (data_point -mean) / std. deviation



Now to define an outlier threshold value is chosen which is generally 3.0. As 99.7% of the data points lie between +/- 3 standard deviation (using Gaussian Distribution approach).

Rows where Z value is greater than 2

#### IQR (Inter-Quartile Range)
Inter Quartile Range approach to finding the outliers is the most commonly used and most trusted approach used in the research field. <Br>
IQR = Quartile3 - Quartile1

To define the outlier base value is defined above and below dataset’s normal range namely Upper and Lower bounds, define the upper and the lower bound (1.5*IQR value is considered) :<br>
upper = Q3 + 1.5 * IQR <br>
lower = Q1 - 1.5 * IQR <br> 

- Removing the outliers:
For removing the outlier, one must follow the same process of removing an entry from the dataset using its exact position in the dataset because in all the above methods of detecting the outliers end result is the list of all those data items that satisfy the outlier definition according to the method used.